In [1]:
import keras

Using TensorFlow backend.


In [2]:
import cv2
import os, gc, sys, glob
import pandas as pd
import numpy as np
from tqdm import tqdm

from sklearn import model_selection
from sklearn import metrics

from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.metrics import categorical_accuracy
from keras.preprocessing.image import ImageDataGenerator

In [3]:
train_set = pd.read_csv('input/train_labels.csv')
test_set = pd.read_csv('input/sample_submission.csv')

In [4]:
train_set.head()

,name,invasive
0,1,0
1,2,0
2,3,1
3,4,0
4,5,1


In [5]:
test_set.head()

,name,invasive
0,1,0.5
1,2,0.5
2,3,0.5
3,4,0.5
4,5,0.5


In [6]:
img_size1 = 224
img_size2 = 224
def read_img(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (img_size1, img_size2))
    return img

train_img, test_img = [], []
for img_path in tqdm(train_set['name'].iloc[: ]):
    train_img.append(read_img('input/train/' + str(img_path) + '.jpg'))
for img_path in tqdm(test_set['name'].iloc[: ]):
    test_img.append(read_img('input/test/' + str(img_path) + '.jpg'))

100%|██████████| 1531/1531 [00:39<00:00, 37.90it/s]


In [7]:
train_img = np.array(train_img, np.float32) / 255
train_label = np.array(train_set['invasive'].iloc[: ])
test_img = np.array(test_img, np.float32) / 255

In [8]:
from keras import backend as K
K.clear_session()
from keras.layers import ZeroPadding2D

def model_nn():
    model = Sequential()
    #model.add(ZeroPadding2D((1,1), input_shape=(img_size1, img_size2, 3)))
    model.add(Conv2D(8, (3, 3), activation='relu', input_shape=(img_size1, img_size2, 3)))
    #model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(8, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))
    
    #model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(16, (3, 3), activation='relu'))
    #model.add(ZeroPadding2D((1,1)))          
    model.add(Conv2D(16, (3, 3), activation='relu'))
    #model.add(ZeroPadding2D((1,1)))
    #model.add(Conv2D(16, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))
    
    #model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    #model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    #model.add(ZeroPadding2D((1,1)))
    #model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))
    
    #model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    #model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(64, (3, 3), activation='relu')) 
    #model.add(ZeroPadding2D((1,1)))
    #model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))
    
    #model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    #model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    #model.add(ZeroPadding2D((1,1)))
    #model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))
    #model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(Flatten())
    #change 2048 to 1024 to speed up
    model.add(Dense(2048, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2048, activation='relu'))
    model.add(Dropout(0.5))

    #model.add(Dense(512, activation='relu'))
    #model.add(Dropout(0.5))
    #model.add(Dense(512, activation='relu'))
    #model.add(Dropout(0.5))
    
    model.add(Dense(1, activation='sigmoid'))
    #change learning rate from 0.001 to 0.002
    sgd = optimizers.SGD(lr = 0.1, decay = 1e-6, momentum = 0.9, nesterov = True) 
    #model.compile(loss = 'binary_crossentropy', optimizer = sgd, metrics=['accuracy'])
    adam = optimizers.Adam(lr = 0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1e-6)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics=['accuracy'])
    
    print(model.summary())
    return model

#change from 8 to 4
n_fold = 6
kf = model_selection.KFold(n_splits = n_fold, shuffle = True)
eval_fun = metrics.roc_auc_score

In [9]:
def run_oof(tr_x, tr_y, te_x, kf):
    preds_train = np.zeros(len(tr_x), dtype = np.float)
    preds_test = np.zeros(len(te_x), dtype = np.float)
    train_loss = []; test_loss = []

    i = 1
    for train_index, test_index in kf.split(tr_x):
    #train_index, test_index = kf.split(tr_x)        
        x_tr = tr_x[train_index]; x_te = tr_x[test_index]
        y_tr = tr_y[train_index]; y_te = tr_y[test_index]

        datagen = ImageDataGenerator(
            # featurewise_center = True,
            rotation_range = 30,
            width_shift_range = 0.2,
            height_shift_range = 0.2,
            # zca_whitening = True,
            shear_range = 0.2,
            zoom_range = 0.2,
            horizontal_flip = True,
            #vertical_flip = True,
            fill_mode = 'nearest')
        datagen.fit(x_tr)

        model = model_nn()
        earlystop = keras.callbacks.EarlyStopping(monitor='val_loss', patience = 30, verbose=0, mode='auto')
        model.fit_generator(datagen.flow(x_tr, y_tr, batch_size = 64),
            validation_data = (x_te, y_te), callbacks = [earlystop],
            steps_per_epoch = len(train_img) / 64, epochs = 250, verbose = 2)

        train_loss.append(eval_fun(y_tr, model.predict(x_tr)[:, 0]))
        test_loss.append(eval_fun(y_te, model.predict(x_te)[:, 0]))

        preds_train[test_index] = model.predict(x_te)[:, 0]
        #if(test_loss[-1] < 0.9) :
        #    break;
        preds_test += model.predict(te_x)[:, 0]

        print('{0}: Train {1:0.5f} Val {2:0.5f}'.format(i, train_loss[-1], test_loss[-1]))
        i += 1
        break

    print('Train: ', train_loss)
    print('Val: ', test_loss)
    print('Train{0:0.5f}_Test{1:0.5f}\n\n'.format(np.mean(train_loss), np.mean(test_loss)))
    #preds_test /= n_fold
    preds_test /= i;
    return preds_train, preds_test

In [10]:
train_pred, test_pred = run_oof(train_img, train_label, test_img, kf)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 222, 222, 8)       224       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 220, 220, 8)       584       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 110, 110, 8)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 108, 108, 16)      1168      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 106, 106, 16)      2320      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 53, 53, 16)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 51, 51, 32)        4640      
__________

17s - loss: 0.1070 - acc: 0.9599 - val_loss: 0.1181 - val_acc: 0.9530
Epoch 64/250
18s - loss: 0.1739 - acc: 0.9322 - val_loss: 0.1282 - val_acc: 0.9582
Epoch 65/250
18s - loss: 0.1189 - acc: 0.9616 - val_loss: 0.1156 - val_acc: 0.9504
Epoch 66/250
17s - loss: 0.1058 - acc: 0.9592 - val_loss: 0.1196 - val_acc: 0.9556
Epoch 67/250
18s - loss: 0.0904 - acc: 0.9635 - val_loss: 0.1025 - val_acc: 0.9556
Epoch 68/250
18s - loss: 0.1036 - acc: 0.9606 - val_loss: 0.0784 - val_acc: 0.9661
Epoch 69/250
17s - loss: 0.1223 - acc: 0.9583 - val_loss: 0.0945 - val_acc: 0.9608
Epoch 70/250
18s - loss: 0.0999 - acc: 0.9632 - val_loss: 0.1103 - val_acc: 0.9661
Epoch 71/250
17s - loss: 0.0879 - acc: 0.9717 - val_loss: 0.0777 - val_acc: 0.9739
Epoch 72/250
18s - loss: 0.1354 - acc: 0.9473 - val_loss: 0.1950 - val_acc: 0.9269
Epoch 73/250
18s - loss: 0.1209 - acc: 0.9577 - val_loss: 0.1359 - val_acc: 0.9608
Epoch 74/250
17s - loss: 0.0939 - acc: 0.9665 - val_loss: 0.1118 - val_acc: 0.9687
Epoch 75/250
17s 

17s - loss: 0.0797 - acc: 0.9699 - val_loss: 0.2224 - val_acc: 0.9060
Epoch 162/250
18s - loss: 0.0714 - acc: 0.9726 - val_loss: 0.1504 - val_acc: 0.9426
Epoch 163/250
17s - loss: 0.0566 - acc: 0.9804 - val_loss: 0.1043 - val_acc: 0.9634
Epoch 164/250
17s - loss: 0.0523 - acc: 0.9825 - val_loss: 0.0750 - val_acc: 0.9582
Epoch 165/250
17s - loss: 0.0822 - acc: 0.9703 - val_loss: 0.0753 - val_acc: 0.9713
Epoch 166/250
18s - loss: 0.0728 - acc: 0.9761 - val_loss: 0.0683 - val_acc: 0.9765
Epoch 167/250
17s - loss: 0.0554 - acc: 0.9839 - val_loss: 0.0663 - val_acc: 0.9739
Epoch 168/250
17s - loss: 0.0615 - acc: 0.9769 - val_loss: 0.1314 - val_acc: 0.9582
Epoch 169/250
18s - loss: 0.0725 - acc: 0.9722 - val_loss: 0.0718 - val_acc: 0.9817
Epoch 170/250
18s - loss: 0.0805 - acc: 0.9689 - val_loss: 0.1052 - val_acc: 0.9634
Epoch 171/250
17s - loss: 0.0510 - acc: 0.9822 - val_loss: 0.0793 - val_acc: 0.9739
Epoch 172/250
17s - loss: 0.0700 - acc: 0.9713 - val_loss: 0.0910 - val_acc: 0.9713
Epoch 

In [11]:
test_set['invasive'] = test_pred
test_set.to_csv('./submit-vgg.csv', index = None)